# Exemplo com Vassoura e dataset PD Behavior

Este notebook demonstra como carregar o dataset `pd_behavior_example.parquet`
 e aplicar a classe `Vassoura`.

In [ ]:
import pandas as pd
from vassoura.core import Vassoura
import vassoura as vs
from audittrail import AuditTrail

pd.set_option('display.max_columns', None)

FILE_PATH_1 = '../../../datasets/lending_club/accepted_2007_to_2018Q4.csv'
FILE_PATH_2 = '../../datasets/lending_club/rejected_2007_to_2018Q4.csv'

In [ ]:
# Carregar dataset de exemplo
def read_and_clean_csv_mixed_types(path, nrows=None, verbose=True):
    """
    Lê um arquivo CSV, detecta colunas com tipos mistos e aplica conversão automática.
    
    Parâmetros:
    - path: caminho para o arquivo CSV
    - nrows: número de linhas a serem lidas (None = todas)
    - verbose: se True, imprime colunas com tipos mistos

    Retorna:
    - DataFrame limpo
    - Dicionário com as colunas que tinham tipos mistos
    """
    # Leitura inicial
    df = pd.read_csv(path, low_memory=False, nrows=nrows)

    # Identificar colunas com tipos mistos
    mixed_type_columns = {}
    for col in df.columns:
        types_in_col = df[col].dropna().apply(type).value_counts()
        if len(types_in_col) > 1:
            mixed_type_columns[col] = types_in_col
            if verbose:
                print(f"\n[!] Coluna '{col}' tem múltiplos tipos:")
                print(types_in_col)

    # Tentativa de padronização
    for col in mixed_type_columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para float.")
        except Exception:
            df[col] = df[col].astype(str)
            if verbose:
                print(f"[✓] Coluna '{col}' convertida para string.")

    return df, mixed_type_columns

df, problemas_1 = read_and_clean_csv_mixed_types(FILE_PATH_1, nrows=1_000)
#loans_rejected, problemas_2 = read_and_clean_csv_mixed_types(FILE_PATH_2, nrows=100_000)

print(df.shape)
display(df.head(3))

# print(loans_rejected.shape)
# display(loans_rejected.head(3))

#### Análise de Target

In [ ]:
df['loan_status'].value_counts(dropna=False, normalize=True) * 100

In [ ]:
TARGET = 'target_risco_credito'

status_de_risco = [
    "Charged Off",
    "Default",
    "Late (31-120 days)",
    #"Late (16-30 days)"
]
df[TARGET] = df["loan_status"].isin(status_de_risco).astype(int)

df[TARGET].value_counts(dropna=False, normalize=True) * 100

### Safras

In [ ]:
#date_col=['issue_d', 'earliest_cr_line','last_pymnt_d','last_credit_pull_d','next_pymnt_d']

temporal_columns = [
    "issue_d",                      # Data de emissão do empréstimo
    "earliest_cr_line",             # Primeira linha de crédito do cliente
    "last_pymnt_d",                 # Último pagamento realizado
    "last_credit_pull_d",           # Última consulta ao crédito
    "next_pymnt_d",                 # Próximo pagamento previsto (se aplicável)
    #"last_pymnt_amnt",             # (associada à data de pagamento)
    "debt_settlement_flag_date",    # Data em que houve acordo de dívida
    "settlement_date"               # Data em que o acordo foi fechado
]


# tratamento para colunas de data
for col in temporal_columns:
    df[col] = pd.to_datetime(df[col], format="%b-%Y")


df["safra"] = df["issue_d"].dt.to_period("M")#.astype(str)  # formato 'YYYY-MM'

print(df["safra"].min())
print(df["safra"].max())

### Captura Inicial

In [ ]:
trail = AuditTrail(
    track_histograms=True,
    track_distributions=True,
    enable_logging=True,
    auto_detect_types=True,
    target_col=TARGET,
    default_keys=["id"]
)

#trail.take_snapshot(df, name="amostra")

In [ ]:
#trail.describe_snapshot("amostra")

#### Limpeza com Vassoura

In [ ]:
assert set(df[TARGET].dropna().unique()) == {0, 1}, "TARGET não é binário!"

In [ ]:
# futuramente podemos pensar no Vassoura como um orquestrador, fazendo diferentes tipos de limpeza para a mesma heuristica
# assim: uma limpeza de correlação no início, um pouco mais 

vs = Vassoura(
    df,
    id_cols = ['id','member_id'],
    date_cols=['safra'],
    ignore_cols=['url']+temporal_columns,
    drop_ignored=True,
    target_col=TARGET,
    verbose='basic',
    engine='pandas',
    
    adaptive_sampling=True,

    process=[
        'missing',
        'variance',
        'scaler'
    ],

    heuristics=[
        "iv",
        "graph_cut",
        "corr",
        "vif",
        'importance',
        'ks_separation',

        # corrigir implementação das heuristicas abaixo
        #"psi_stability",
        #"perm_importance_lgbm",
        #"partial_corr_cluster",
        #"drift_vs_target_leakage",
    ],
    
    params={
        "missing": 0.60,
        "corr": 0.85,
        "vif": 10,
        "iv": 0.01,
        "graph_cut": 0.9,
    },
    n_steps=2,         # controla quantas remoções por iteração
    vif_n_steps=2      # VIF será sempre 1-step (a menos que você altere)
)

df_limpo = vs.run(recompute=True)

In [ ]:
print(df_limpo.shape)
df_limpo.head()

In [ ]:
vs.generate_report("../reports/vassoura_report.html")